# Assignment 5

Ten Kamilla, BDA-1902

In this assignment krysha.kz website was parsed using Selenium. More precisely, code above demonstrates redirecting to krysha.kz, choosing Almaty region option and retrieving information about the appartment for sale announcements from all links on 7 pages. Moreover, some data cleaning and visualization are implemented.

Firstly, installing and importing all required modules 

In [2]:
!pip install selenium
!pip install webdriver_manager 

In [3]:
from selenium import webdriver
#chrome browser is used
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import Select

In [4]:
#chrome webdriver installation
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\Kamilla\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


### Data scraping

The following code is for gathering announcement links.

In [145]:
#list for announcement links
links = []
#list for each of 7 pages with announcements
pages = []
#instance of webdriver
driver = webdriver.Chrome(executable_path = r'C:\Users\Kamilla\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe')
driver.maximize_window()
driver.get('https://krisha.kz')
#clicking to drop down for regions
region_searh = driver.find_element_by_id('region-selected-value').click();
#waiting
sleep(3);
#choosing Almaty
dropCity = Select(driver.find_element_by_xpath("//div[@class='element-region-dropdown-selects']/div/select")).select_by_value("2");
sleep(3)
#clicking choose button
select_button = driver.find_element_by_xpath("//div[@class='region-dropdown-actions']/a[@class='btn btn-primary region-dropdown-action region-dropdown-action-apply']").click();
sleep(3)
#clicking search button
search_button = driver.find_element_by_class_name('search-btn-main').click();
sleep(3)
#retrieving all 7 page links
for i in range(1, 8):
    page = driver.current_url + '?page='+str(i);
    pages.append(page)
#in page links gathering announcement links
for p in pages:
    driver.get(p)
    one_page_links = driver.find_elements_by_class_name('a-card__image')
    for o in one_page_links:
        link = o.get_attribute('href')
        links.append(link)
driver.close()   

In [147]:
#overall, 143 announcements were found
print(len(links))

143


In [111]:
#function to clean data(price)
def clean(text, symbols=['\n']):
    for symbol in symbols:
        text = text.replace(symbol, '')
    return text.strip()

In [151]:
#importing exception module for try,except block
from selenium.common.exceptions import NoSuchElementException
#list for each announcement info
infos = []
#retrieving name, phone, city, price, author for each link
for l in links:
    driver1 = webdriver.Chrome(executable_path = r'C:\Users\Kamilla\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe')
    driver1.maximize_window()
    driver1.get(l)
    #showing hidden part of phone number
    phone_click = driver1.find_element_by_class_name('show-phones').click()
    name = driver1.find_element_by_tag_name('h1').text
    price = driver1.find_element_by_class_name('offer__price').text
    #cleaning price
    price = clean(price, ['\n', 'от', '〒', ' '])
    price = pd.to_numeric(price)
    city = driver1.find_element_by_xpath("//div[@class='offer__location offer__advert-short-info']/span").text
    author = driver1.find_element_by_class_name('owners__name').text
    phone = driver1.find_element_by_class_name('offer__contacts-phones').text
    #some announcements do not have images
    try:
        img_url = driver1.find_element_by_xpath("//div[@class='gallary__main']/a").get_attribute('href')
    except NoSuchElementException:
            img_url = 'no image'
    info = {'name':name, 'price':price, 'city':city, 'author': author, 'phone':phone, 'img_url':img_url}
    infos.append(info)
    driver1.close()

At this moment, all reqiured data is scraped. 

### Data cleaning 

In [152]:
data =pd.DataFrame(infos)

In [155]:
data.head()

,name,price,city,author,phone,img_url
0,"3-комнатная квартира, 174 м², 4/6 этаж, мкр Ер...",71500000,"Алматы, Бостандыкский р-н",ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/1f/1fa22b13-ed4...
1,"1-комнатная квартира, 44 м², 1/9 этаж, мкр Шуг...",16900000,"Алматы, Наурызбайский р-н",ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/d8/d8edf96f-aaa...
2,"1-комнатная квартира, 40 м², 7/9 этаж, мкр Шуг...",15700000,"Алматы, Наурызбайский р-н",ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/5e/5ef0d303-43d...
3,"4-комнатная квартира, 100 м², 3/3 этаж, Жулдыз...",26000000,"Алматы, Турксибский р-н",Алтынай Алимкулова,+7 707 457 1994\n+7 778 583 2676,https://photos-kr.kcdn.kz/webp/76/761549e1-321...
4,"1-комнатная квартира, 123 м², 7/9 этаж, мкр Же...",20000000,"Алматы, Ауэзовский р-н",Хозяин недвижимости,+7 708 189 2433,no image


From the dataframe above, the city column was splitted to two - city and district. 

In [156]:
data[['city','district']] = data.city.str.split(",",expand=True,)
data.head()

,name,price,city,author,phone,img_url,district
0,"3-комнатная квартира, 174 м², 4/6 этаж, мкр Ер...",71500000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/1f/1fa22b13-ed4...,Бостандыкский р-н
1,"1-комнатная квартира, 44 м², 1/9 этаж, мкр Шуг...",16900000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/d8/d8edf96f-aaa...,Наурызбайский р-н
2,"1-комнатная квартира, 40 м², 7/9 этаж, мкр Шуг...",15700000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/5e/5ef0d303-43d...,Наурызбайский р-н
3,"4-комнатная квартира, 100 м², 3/3 этаж, Жулдыз...",26000000,Алматы,Алтынай Алимкулова,+7 707 457 1994\n+7 778 583 2676,https://photos-kr.kcdn.kz/webp/76/761549e1-321...,Турксибский р-н
4,"1-комнатная квартира, 123 м², 7/9 этаж, мкр Же...",20000000,Алматы,Хозяин недвижимости,+7 708 189 2433,no image,Ауэзовский р-н


In [162]:
#to find area of each appartment, we take the second item from string splitted by commas
data['area'] = data['name'].str.rsplit(',').str[1] 
data.head()

,name,price,city,author,phone,img_url,district,area
0,"3-комнатная квартира, 174 м², 4/6 этаж, мкр Ер...",71500000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/1f/1fa22b13-ed4...,Бостандыкский р-н,174 м²
1,"1-комнатная квартира, 44 м², 1/9 этаж, мкр Шуг...",16900000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/d8/d8edf96f-aaa...,Наурызбайский р-н,44 м²
2,"1-комнатная квартира, 40 м², 7/9 этаж, мкр Шуг...",15700000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/5e/5ef0d303-43d...,Наурызбайский р-н,40 м²
3,"4-комнатная квартира, 100 м², 3/3 этаж, Жулдыз...",26000000,Алматы,Алтынай Алимкулова,+7 707 457 1994\n+7 778 583 2676,https://photos-kr.kcdn.kz/webp/76/761549e1-321...,Турксибский р-н,100 м²
4,"1-комнатная квартира, 123 м², 7/9 этаж, мкр Же...",20000000,Алматы,Хозяин недвижимости,+7 708 189 2433,no image,Ауэзовский р-н,123 м²


In [163]:
#number of rooms is the first character in the name column
data['roomsNumber'] = data['name'].str[0] 
data.head()

,name,price,city,author,phone,img_url,district,area,roomsNumber
0,"3-комнатная квартира, 174 м², 4/6 этаж, мкр Ер...",71500000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/1f/1fa22b13-ed4...,Бостандыкский р-н,174 м²,3
1,"1-комнатная квартира, 44 м², 1/9 этаж, мкр Шуг...",16900000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/d8/d8edf96f-aaa...,Наурызбайский р-н,44 м²,1
2,"1-комнатная квартира, 40 м², 7/9 этаж, мкр Шуг...",15700000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/5e/5ef0d303-43d...,Наурызбайский р-н,40 м²,1
3,"4-комнатная квартира, 100 м², 3/3 этаж, Жулдыз...",26000000,Алматы,Алтынай Алимкулова,+7 707 457 1994\n+7 778 583 2676,https://photos-kr.kcdn.kz/webp/76/761549e1-321...,Турксибский р-н,100 м²,4
4,"1-комнатная квартира, 123 м², 7/9 этаж, мкр Же...",20000000,Алматы,Хозяин недвижимости,+7 708 189 2433,no image,Ауэзовский р-н,123 м²,1


In [238]:
#converting rooms number to int
data['roomsNumber'] =pd.to_numeric(data['roomsNumber'])

In [171]:
#deleting whitespaces and units, converting to float
data['area'] = data['area'].str.replace(' ', '')
data['area'] = data['area'].str.replace('м²', '')
data['area'] =pd.to_numeric(data['area'])
data.head()

,name,price,city,author,phone,img_url,district,area,roomsNumber
0,"3-комнатная квартира, 174 м², 4/6 этаж, мкр Ер...",71500000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/1f/1fa22b13-ed4...,Бостандыкский р-н,174.0,3
1,"1-комнатная квартира, 44 м², 1/9 этаж, мкр Шуг...",16900000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/d8/d8edf96f-aaa...,Наурызбайский р-н,44.0,1
2,"1-комнатная квартира, 40 м², 7/9 этаж, мкр Шуг...",15700000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/5e/5ef0d303-43d...,Наурызбайский р-н,40.0,1
3,"4-комнатная квартира, 100 м², 3/3 этаж, Жулдыз...",26000000,Алматы,Алтынай Алимкулова,+7 707 457 1994\n+7 778 583 2676,https://photos-kr.kcdn.kz/webp/76/761549e1-321...,Турксибский р-н,100.0,4
4,"1-комнатная квартира, 123 м², 7/9 этаж, мкр Же...",20000000,Алматы,Хозяин недвижимости,+7 708 189 2433,no image,Ауэзовский р-н,123.0,1


In [173]:
#for rows with several numbers
data['phone'] = data['phone'].str.replace('\n', ' ')
data.head()

,name,price,city,author,phone,img_url,district,area,roomsNumber
0,"3-комнатная квартира, 174 м², 4/6 этаж, мкр Ер...",71500000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/1f/1fa22b13-ed4...,Бостандыкский р-н,174.0,3
1,"1-комнатная квартира, 44 м², 1/9 этаж, мкр Шуг...",16900000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/d8/d8edf96f-aaa...,Наурызбайский р-н,44.0,1
2,"1-комнатная квартира, 40 м², 7/9 этаж, мкр Шуг...",15700000,Алматы,ТANA REALTY,+7 700 183 8311,https://photos-kr.kcdn.kz/webp/5e/5ef0d303-43d...,Наурызбайский р-н,40.0,1
3,"4-комнатная квартира, 100 м², 3/3 этаж, Жулдыз...",26000000,Алматы,Алтынай Алимкулова,+7 707 457 1994 +7 778 583 2676,https://photos-kr.kcdn.kz/webp/76/761549e1-321...,Турксибский р-н,100.0,4
4,"1-комнатная квартира, 123 м², 7/9 этаж, мкр Же...",20000000,Алматы,Хозяин недвижимости,+7 708 189 2433,no image,Ауэзовский р-н,123.0,1


In [175]:
#saving data to excel
data.to_excel('TenKamillaBDA1902_hw5krysha.xlsx', index = 'false')

In [239]:
data.describe()

,price,area,roomsNumber
count,1.430000e+02,143.000000,143.000000
mean,5.299721e+07,89.426294,2.503497
std,9.787205e+07,74.097773,1.086888
min,6.999000e+06,17.400000,1.000000
25%,2.500000e+07,53.000000,2.000000
50%,3.339000e+07,68.000000,2.000000
75%,5.475000e+07,101.200000,3.000000
max,1.128000e+09,750.000000,6.000000


The above statistics shows that mean price is 50 millions, mean area is 89 square meters and mean rooms number is 2.5. The cheapest appartment in his sample has 17 square meters and 1 room, it costs about 7 millions. While, the most expensive one has six rooms, 759 square meters and costs more than 1 billion tenge.

### Visualization

It is important to mention that all further visualizations and conclusions are true only for previously scrapped sampled and do not reflect real world situation.

In [178]:
import plotly.express as px 
distr = px.histogram(data, x= 'district', width=600, height=400, title="Distribution of appartments by district")
distr.show();

The histogram above shows the distribution of appartments for sale in different Almaty districts. It is clear that the most popular one is "Бостандыкский", then "Алмалинский" and "Медеуский". The least popular ones are "Наурызбайский" and "Жетысуйский".

In [189]:
fig = px.strip(data, x="price", y="district", width=800, height=500, title="District influence on Price")
fig.show();

The categorical plot illustrates that the distrit with highest price for appartments is "Медеуский".While, overall, the price amoung all districts is no more than 0.2 billions

In [236]:
fig = px.scatter(data, x="area", y="price", title = "Price vs. Area")
fig.show()

The scatter plot shows positive correlation trend between area and price, thus it supports a common opinion about their connection.

### Conclusion

Scrapped during this assignment data allowed to perform some descritive statistics and visualize. In conclusion, basing on this sample, it is more likely to find appartment in "Бостандыкский" district. For those who look for the cheapest option "Наурызбайский" and "Жетысуйский"are the best. The increase in area provokes increase in price pattern was also shown.